<a href="https://colab.research.google.com/github/zasuo-michikusa/Google_Colab_StableDiffusion/blob/main/StableDiffusion_AnimateDiff_custom_ipynb_20240103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI（Forge）」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【06】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【05】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【05】の（▶）を押し③と同様

⑥生成したデータを保存する場合は【06】の（▶）を実行

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_AnimateDiff環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [2]:
# Stable Diffusion Web-UI Forge のクローン（インストール）
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge

# Styles-Editorのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/chrisgoringe/Styles-Editor.git
%cd /content/stable-diffusion-webui-forge

# adetailerのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Bing-su/adetailer.git
%cd /content/stable-diffusion-webui-forge

# aspect-ratio-helperのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git
%cd /content/stable-diffusion-webui-forge

# prompt-all-in-oneのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
%cd /content/stable-diffusion-webui-forge

# ultimate-upscale-for-automatic1111のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
%cd /content/stable-diffusion-webui-forge

# SegmentAnythingのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-webui-segment-anything.git
%cd /content/stable-diffusion-webui-forge

# AnimateDiff（Forge版）のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-forge-animatediff.git /content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff
%cd /content/stable-diffusion-webui-forge

# Deforumのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/deforum-art/sd-webui-deforum.git
%cd /content/stable-diffusion-webui-forge


Cloning into 'stable-diffusion-webui-forge'...
remote: Enumerating objects: 37893, done.
remote: Counting objects: 100% (2734/2734), done.
remote: Compressing objects: 100% (803/803), done.
remote: Total 37893 (delta 2030), reused 2467 (delta 1888), pack-reused 35159
Receiving objects: 100% (37893/37893), 43.94 MiB | 32.82 MiB/s, done.
Resolving deltas: 100% (25931/25931), done.
/content/stable-diffusion-webui-forge
/content/stable-diffusion-webui-forge/extensions
Cloning into 'Styles-Editor'...
remote: Enumerating objects: 739, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 739 (delta 112), reused 104 (delta 87), pack-reused 516
Receiving objects: 100% (739/739), 143.52 KiB | 2.76 MiB/s, done.
Resolving deltas: 100% (389/389), done.
/content/stable-diffusion-webui-forge
/content/stable-diffusion-webui-forge/extensions
Cloning into 'adetailer'...
remote: Enumerating objects: 2544, done.
remote: Counting objects: 10

👇【03】MP4形式の出力用に「ffpeg」をインストール

In [3]:
%cd /content/stable-diffusion-webui-forge/
!venv\Scripts\activate
!pip install imageio[ffmpeg]

/content/stable-diffusion-webui-forge
/bin/bash: line 1: venvScriptsactivate: command not found


👇【04】GoogleドライブへHugingFaceからDL

「AnimateDiff」用の「MotionModel」

「SegmentAnything」用の「sam」「GroundingDINO」

In [4]:
%cd /content/stable-diffusion-webui-forge/

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/sam

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinB.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinT_OGC.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swint_ogc.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swinb_cogcoor.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

/content/stable-diffusion-webui-forge
--2024-04-27 22:12:26--  https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/36/eb3682f47ef4d8ca29a706b4dac4d42561a0670a17f803febf9feb06ed6286ea/dc16214feab8b0d8b9e7b05f5a4d9c5e2648601e9c807c51b1b0dc7478879535?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mm_sd14_v1.safetensors%3B+filename%3D%22mm_sd14_v1.safetensors%22%3B&Expires=1714515146&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDUxNTE0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViLzM2L2ViMzY4MmY0N2VmNGQ4Y2EyOWE3MDZiNGRhYzRkNDI1NjFhMDY3MGExN2Y4MDNmZWJmOWZlYjA2ZWQ2Mjg2ZWEvZ

👇【05】WEB-UIの起動（各種大容量データはGoogleドライブを参照する設定） ※起動オプション追記もお好みで可能

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。

In [ ]:
!python launch.py --share --no-half-vae --disable-nan-check\
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version: f0.0.17v1.8.0rc-latest-276-g29be1da7
Commit hash: 29be1da7cf2b5dccfc70fbdd33eb35c56a31ffb7
Installing clip
Installing open_clip
Cloning assets into /content/stable-diffusion-webui-forge/repositories/stable-diffusion-webui-assets...
Cloning into '/content/stable-diffusion-webui-forge/repositories/stable-diffusion-webui-assets'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 0), reused 20 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 132.70 KiB | 1.77 MiB/s, done.
Cloning Stable Diffusion into /content/stable-diffusion-webui-forge/repositories/stable-diffusion-stability-ai...
Cloning into '/content/stable-diffusion-webui-forge/repositories/stable-diffusion-stability-ai'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (306/3

👇【06】いったん【05】の処理を停止させた後に（▶）を実行してください。

生成したデータ「Outputデータ」をGoogleドライブへコピーします。

≪注意≫フォルダごと追加するので、フォルダ構成が多重化していきます。

※こまめにGoogleドライブ内の「outputs」フォルダのデータをローカルに移動と削除をしてください。

In [ ]:
!cp -r "/content/stable-diffusion-webui-forge/outputs/txt2img-grids" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids"
!cp -r "/content/stable-diffusion-webui-forge/outputs/txt2img-images" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images"